## GRE walkthrough

This jupyter notebook is a walkthrough to construct a Gradient Recalled Echo (GRE) pulse sequence.



1. First, import the necessary packages.

In [ ]:
import math

import numpy as np

from pypulseq.Sequence.sequence import Sequence
from pypulseq.calc_duration import calc_duration
from pypulseq.make_adc import make_adc
from pypulseq.make_delay import make_delay
from pypulseq.make_sinc_pulse import make_sinc_pulse
from pypulseq.make_trap_pulse import make_trapezoid
from pypulseq.opts import Opts

2. Construct a `Sequence` object. A `Sequence` object is analogous to a pulse sequence. The following steps will demonstrate creating pulse sequence events and adding them to this `Sequence` object. The MR scanner's hardware limits are defined in an `Opts` object.

In [ ]:
seq = Sequence()
fov = 256e-3 # field of view
Nx = 256 # number of frequency encodes
Ny = 256 # number of phase encodes
alpha = 10 # RF flip
slice_thickness = 3e-3
TE = 7.38e-3 # echo time
TR = 100e-3 # repetition time

rf_spoiling_inc = 117

sys = Opts(max_grad=28, grad_unit='mT/m', max_slew=150, slew_unit='T/m/s', rf_ringdown_time=20e-6, rf_dead_time=100e-6,
           adc_dead_time=10e-6)

3. Now, construct the radio-frequency (RF), gradient, delay and ADC readout events.

In [ ]:
rf, gz, gzr = make_sinc_pulse(flip_angle=alpha * math.pi / 180, duration=4e-3, slice_thickness=slice_thickness,
                              apodization=0.5, time_bw_product=4, system=sys)

delta_k = 1 / fov
gx = make_trapezoid(channel='x', flat_area=Nx * delta_k, flat_time=6.4e-3, system=sys)
adc = make_adc(num_samples=Nx, duration=gx.flat_time, delay=gx.rise_time, system=sys)
gx_pre = make_trapezoid(channel='x', area=-gx.area / 2, duration=2e-3, system=sys)
gz_reph = make_trapezoid(channel='z', area=-gz.area / 2, duration=2e-3, system=sys)
phase_areas = (np.arange(Ny) - Ny / 2) * delta_k

gx_spoil = make_trapezoid(channel='x', area=2 * Nx * delta_k, system=sys)
gz_spoil = make_trapezoid(channel='z', area=4 / slice_thickness, system=sys)

delay_TE = math.ceil((TE - calc_duration(gx_pre) - gz.fall_time - gz.flat_time / 2 - calc_duration(
    gx) / 2) / seq.grad_raster_time) * seq.grad_raster_time
delay_TR = math.ceil((TR - calc_duration(gx_pre) - calc_duration(gz) - calc_duration(
    gx) - delay_TE) / seq.grad_raster_time) * seq.grad_raster_time

assert np.all(delay_TR >= calc_duration(gx_spoil, gz_spoil))

4. Add the constructed pulse sequence events to the `Sequence` object created earlier.

The functionality of the `add_block()` method can be tested by adding only one event to the `Sequence` object and obtaining the duration of the added events. For example, add only the `rf` object to `seq`. Then, call `seq.duration()` to obtain the duration of all the events added to `seq` (in this case, only the `rf` event so far).

In [ ]:
rf_phase = 0
rf_inc = 0

for i in range(Ny): # We have Ny phase encodes
    rf.phase_offset = rf_phase / 180 * np.pi
    adc.phase_offset = rf_phase / 180 * np.pi
    rf_inc = divmod(rf_inc + rf_spoiling_inc, 360.0)[1]
    rf_phase = divmod(rf_phase + rf_inc, 360.0)[1]

    seq.add_block(rf, gz)
    gy_pre = make_trapezoid(channel='y', area=phase_areas[i], duration=2e-3, system=sys)
    seq.add_block(gx_pre, gy_pre, gz_reph)
    seq.add_block(make_delay(delay_TE))
    seq.add_block(gx, adc)
    gy_pre.amplitude = -gy_pre.amplitude
    seq.add_block(make_delay(delay_TR), gx_spoil, gy_pre, gz_spoil)

5. Visualise the constructed pulse sequence by calling the `plot()` command. The `plot()` command visualises the ADC and RF events in one window, and the gradient events in another window. For GRE, the `plot()` command should display two plots that look like this:

![GRE Plot 1](./gre_1.png)
![GRE Plot 2](./gre_2.png)

In [ ]:
seq.plot()

6. Export the pulse sequence as a `.seq` file. This file can be executed on MR scanners (Siemens/[GE](https://toppemri.github.io)/
[Bruker](https://github.com/pulseq/bruker_interpreter)) to acquire data.

In [ ]:
seq.write('gre_pypulseq.seq')